## Predicting Spectrum Traits Grid Search

## Working version

In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
import HyperSpectrumDataSet as hsd
import torch.optim as optim
import HyperSpectrumModel as models
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter
import sys
import datetime
import copy

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("... runing on .. ", device)

... runing on ..  cuda


In [3]:
exp_desc = "HyperSpectrumModelAvgPoolDeepCNN_V_GridSearch"

now = datetime.datetime.now()

script_dir = os.path.dirname("__file__")

result_file_name = os.path.join(script_dir, "results")
result_file_name = os.path.join(result_file_name, exp_desc + "_" + now.strftime("%Y-%m-%d-%H-%M")+"_results.txt")

log_dir = os.path.join(script_dir, "runs")
log_dir = os.path.join(log_dir, exp_desc + "_" + now.strftime("%Y-%m-%d-%H-%M"))


writer = SummaryWriter(log_dir=log_dir)

In [4]:
def nn_setup(structure='model1',dropout=0,lr = 0.001, weight_decay=0, stddev=0.):
    
    if structure == 'model1':
        model = models.HyperSpectrumModelAvgPoolDeepCNN_V(dropout_percent=dropout, stddev = stddev) 
    else:
        print("Im sorry but {} is not a valid model.Did you mean model1?".format(structure))
        
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, 
                           weight_decay=weight_decay, amsgrad=False)
    model.to(device)
    return model, criterion, optimizer 
          

In [5]:
def load_ds(trait="", dataset="train", batch_size = 32):
    ds = hsd.HyperSpectrumDataSet(trait, dataset)
    ds_loader = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)
    
    return ds, ds_loader

In [6]:
def compute_label_sq_diff(dataset):
    sum = 0
    for i in range(len(dataset)):
        values, labels = dataset[i]
        sum += labels

    label_mean = sum / len(dataset)
    #print("train label mean: " + str(label_mean))

    label_sq_diff = 0
    for i in range(len(dataset)):
        values, labels = dataset[i]
        label_sq_diff += (labels -  label_mean) **2
    
    return label_sq_diff

In [7]:
all_traits = ["LMA_O", "Narea_O", "SPAD_O", "Nmass_O", "Pmass_O", "Vcmax", "Vcmax25", "J", "Photo_O", "Cond_O"]

In [8]:
run_traits = ["Vcmax", "Vcmax25", "J", "Photo_O", "Cond_O"]

In [9]:
batch_size = 64

## Train model

In [10]:
def checkPerform(model, dataset, dsloader):
    label_sq_diff = compute_label_sq_diff(dataset)
    pred_sq_diff = 0.0
    num_iters = 0
    running_loss = 0.0
    model.eval()
    for i, data in enumerate(dsloader, 0):
        values, labels = data
        labels = labels.unsqueeze(1)
        values = values.to(device)
        labels = labels.to(device)
        outputs = model(values)
        num_iters += 1
        diff = outputs - labels
        pred_sq_diff += torch.sum(torch.mul(diff, diff), 0).detach().cpu().numpy()

    R_square_value = 1 - pred_sq_diff / label_sq_diff
    
    return R_square_value

In [11]:
def train_model(trait, structure="HyperSpectrumModelAvgPoolDeepCNN_V", 
                dropout=0.3, lr=0.001, 
                weight_decay=0., stddev=0.001):    
    num_epochs = 1000
    test_every = 10
    bestModels = {}
    train_results = {}
    train_ds, trainloader = load_ds(trait, "train", batch_size)    
    val_ds, validationloader = load_ds(trait, "val", batch_size)
    test_ds, testloader = load_ds(trait, "test", batch_size)
    
    model, criterion, optimizer = nn_setup(structure=structure, dropout=dropout, 
                                           lr=lr, weight_decay=weight_decay, stddev = stddev)
    print(model)
    train_label_sq_diff = compute_label_sq_diff(train_ds)

    bestModel = copy.deepcopy(model)
    best_validation_rsquare = -99999.99
    train_R_square_value = 0
    not_improved_count = 0
    early_stoppings = 20
    for epoch in range(num_epochs):
        running_loss = 0.0
        pred_sq_diff = 0.0
        model.train()
        for i, data in enumerate(trainloader, 0):
            values, labels = data
            labels = labels.unsqueeze(1)
            values = values.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(values)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            diff = outputs - labels
            pred_sq_diff += torch.sum(torch.mul(diff, diff),0).detach().cpu().numpy()

        train_R_square_value = 1 - pred_sq_diff / train_label_sq_diff
        if((epoch % test_every) == 0):
            print("Epoch: ", epoch, " Trait: ", target_trait, ", Train_R_square_value: ", np.asscalar(train_R_square_value))

        if ((epoch % test_every) == 0):
            val_R_square_value = checkPerform(model, val_ds, validationloader)

            writer.add_scalar(target_trait + '/training_Rsquare', np.asscalar(train_R_square_value), epoch)
            writer.add_scalar(target_trait + '/validation_Rsquare', np.asscalar(val_R_square_value), epoch)
            print("validation rsquare: " + str(np.asscalar(val_R_square_value)))
            # copy the best model so far
            if (best_validation_rsquare < val_R_square_value):
                print("copying best model")
                bestModel = copy.deepcopy(model)
                best_validation_rsquare = val_R_square_value
                not_improved_count =0
            else:
                not_improved_count += 1
                if not_improved_count == early_stoppings:
                    print ("model is not improved in %d" % early_stoppings)
                    break
    
    print("best_validation_rsquare " + str(np.asscalar(best_validation_rsquare)))    
    return bestModel, best_validation_rsquare

## grid search: learning rate

In [29]:
params = {
    'lr': [0.0001, 0.0005, 0.001, 0.005, 0.01], # default 0.001
}

dropout_percent = 0
weight_decay = 0
stddev = 0

bestModels = {}
train_results = {}
bestParams = {}
for target_trait in run_traits:
    best_validation_rsquare = -99999.99
    best_lr, best_dropout, best_weight_decay, best_stddev = 0., 0., 0., 0.0
    for lr in params['lr']:
        print("********** grid search ********** ")
        print ("lr: " + str(lr))
        print("********************************* ")
        trained_model, validation_rsquare = train_model(trait=target_trait, structure="model1", 
                                                 dropout=dropout, lr=lr, 
                                                 weight_decay=weight_decay,
                                                 stddev=stddev)
        if best_validation_rsquare < validation_rsquare:
            best_model = copy.deepcopy(trained_model)
            best_validation_rsquare = validation_rsquare
            best_lr, best_dropout, best_weight_decay,best_stddev = lr, dropout, weight_decay, stddev
                    
    print("gridsearch: best_validation_rsquare: " + str(np.asscalar(best_validation_rsquare)))
    print("Best lr: ", best_lr)
    bestParams[target_trait] = {}
    bestParams[target_trait]['lr'] = best_lr
    bestModels[target_trait] = copy.deepcopy(best_model)
    train_results[target_trait] =  best_validation_rsquare

********** grid search ********** 
lr: 0.0001
********************************* 
HyperSpectrumModelAvgPoolDeepCNN_V(
  (noise): GaussianNoise()
  (avgPool1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (block1): DeepBasicBlock(
    (Conv1D): Conv1d(1, 50, kernel_size=(3,), stride=(3,), padding=(1,))
    (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block2_1): DeepBasicBlock(
    (Conv1D): Conv1d(50, 100, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block2_2): DeepBasicBlock(
    (Conv1D): Conv1d(100, 200, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block3_1): DeepBasicBlock(
    (Conv1D): Conv1d(200, 400, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(400, eps=1e-05, momentum

Epoch:  70  Trait:  LMA_O , Train_R_square_value:  0.8551615476608276
validation rsquare: 0.8530101180076599
copying best model
Epoch:  80  Trait:  LMA_O , Train_R_square_value:  0.8568036556243896
validation rsquare: 0.7822256088256836
Epoch:  90  Trait:  LMA_O , Train_R_square_value:  0.876228392124176
validation rsquare: 0.8454071283340454
Epoch:  100  Trait:  LMA_O , Train_R_square_value:  0.8457545638084412
validation rsquare: 0.870682954788208
copying best model
Epoch:  110  Trait:  LMA_O , Train_R_square_value:  0.8499600887298584
validation rsquare: 0.8302943706512451
Epoch:  120  Trait:  LMA_O , Train_R_square_value:  0.8832156658172607
validation rsquare: 0.8511008620262146
Epoch:  130  Trait:  LMA_O , Train_R_square_value:  0.9003128409385681
validation rsquare: 0.8307024836540222
Epoch:  140  Trait:  LMA_O , Train_R_square_value:  0.8931701183319092
validation rsquare: 0.8555209636688232
Epoch:  150  Trait:  LMA_O , Train_R_square_value:  0.8947990536689758
validation rsqua

Epoch:  360  Trait:  LMA_O , Train_R_square_value:  0.8976176977157593
validation rsquare: 0.5954858064651489
Epoch:  370  Trait:  LMA_O , Train_R_square_value:  0.9041554927825928
validation rsquare: 0.7822220325469971
Epoch:  380  Trait:  LMA_O , Train_R_square_value:  0.90068519115448
validation rsquare: 0.1621071696281433
Epoch:  390  Trait:  LMA_O , Train_R_square_value:  0.9101564884185791
validation rsquare: 0.7771672010421753
Epoch:  400  Trait:  LMA_O , Train_R_square_value:  0.9143893718719482
validation rsquare: 0.7096794843673706
Epoch:  410  Trait:  LMA_O , Train_R_square_value:  0.8995646238327026
validation rsquare: 0.7184451222419739
model is not improved in 20
best_validation_rsquare 0.8586159944534302
********** grid search ********** 
lr: 0.005
********************************* 
HyperSpectrumModelAvgPoolDeepCNN_V(
  (noise): GaussianNoise()
  (avgPool1): AvgPool1d(kernel_size=(3,), stride=(3,), padding=(0,))
  (block1): DeepBasicBlock(
    (Conv1D): Conv1d(1, 50, ker

Epoch:  60  Trait:  LMA_O , Train_R_square_value:  -0.030521035194396973
validation rsquare: -0.0018616914749145508
Epoch:  70  Trait:  LMA_O , Train_R_square_value:  -0.13608324527740479
validation rsquare: -0.0229417085647583
Epoch:  80  Trait:  LMA_O , Train_R_square_value:  -0.04507768154144287
validation rsquare: -0.028397202491760254
Epoch:  90  Trait:  LMA_O , Train_R_square_value:  -0.030362606048583984
validation rsquare: -0.003667473793029785
Epoch:  100  Trait:  LMA_O , Train_R_square_value:  -0.04865133762359619
validation rsquare: -0.006422996520996094
Epoch:  110  Trait:  LMA_O , Train_R_square_value:  -0.037090301513671875
validation rsquare: -0.000605463981628418
Epoch:  120  Trait:  LMA_O , Train_R_square_value:  -0.0960698127746582
validation rsquare: -0.02166008949279785
Epoch:  130  Trait:  LMA_O , Train_R_square_value:  -0.045706987380981445
validation rsquare: -0.037285804748535156
Epoch:  140  Trait:  LMA_O , Train_R_square_value:  -0.10878050327301025
validation

Epoch:  290  Trait:  Narea_O , Train_R_square_value:  0.9565200805664062
validation rsquare: 0.8895033001899719
Epoch:  300  Trait:  Narea_O , Train_R_square_value:  0.9541571736335754
validation rsquare: 0.9020959138870239
Epoch:  310  Trait:  Narea_O , Train_R_square_value:  0.94881671667099
validation rsquare: 0.8934154510498047
Epoch:  320  Trait:  Narea_O , Train_R_square_value:  0.9494847655296326
validation rsquare: 0.894895076751709
Epoch:  330  Trait:  Narea_O , Train_R_square_value:  0.9588294625282288
validation rsquare: 0.8880647420883179
Epoch:  340  Trait:  Narea_O , Train_R_square_value:  0.9559298753738403
validation rsquare: 0.908290445804596
Epoch:  350  Trait:  Narea_O , Train_R_square_value:  0.9493574500083923
validation rsquare: 0.8990024924278259
Epoch:  360  Trait:  Narea_O , Train_R_square_value:  0.9543842673301697
validation rsquare: 0.8906832933425903
Epoch:  370  Trait:  Narea_O , Train_R_square_value:  0.9432713985443115
validation rsquare: 0.9018654227256

...

Epoch:  40  Trait:  Cond_O , Train_R_square_value:  0.4706714153289795
validation rsquare: 0.17196232080459595
Epoch:  50  Trait:  Cond_O , Train_R_square_value:  0.5335684418678284
validation rsquare: 0.20144683122634888
copying best model
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  0.565734326839447
validation rsquare: 0.15392935276031494
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  0.601824939250946
validation rsquare: 0.21166908740997314
copying best model
Epoch:  80  Trait:  Cond_O , Train_R_square_value:  0.593935489654541
validation rsquare: 0.2336738109588623
copying best model
Epoch:  90  Trait:  Cond_O , Train_R_square_value:  0.646053671836853
validation rsquare: 0.28261232376098633
copying best model
Epoch:  100  Trait:  Cond_O , Train_R_square_value:  0.6503820419311523
validation rsquare: 0.17210179567337036
Epoch:  110  Trait:  Cond_O , Train_R_square_value:  0.6089525818824768
validation rsquare: -0.029458284378051758
Epoch:  120  Trait:  Cond_O , Train_R

Epoch:  30  Trait:  Cond_O , Train_R_square_value:  0.429969847202301
validation rsquare: 0.240362286567688
copying best model
Epoch:  40  Trait:  Cond_O , Train_R_square_value:  0.5533958673477173
validation rsquare: 0.11876523494720459
Epoch:  50  Trait:  Cond_O , Train_R_square_value:  0.5480415225028992
validation rsquare: 0.1527387499809265
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  0.5646705031394958
validation rsquare: -0.47879183292388916
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  0.5674461126327515
validation rsquare: 0.05919092893600464
Epoch:  80  Trait:  Cond_O , Train_R_square_value:  0.537917971611023
validation rsquare: 0.252935528755188
copying best model
Epoch:  90  Trait:  Cond_O , Train_R_square_value:  0.607118546962738
validation rsquare: -0.32406866550445557
Epoch:  100  Trait:  Cond_O , Train_R_square_value:  0.6182190179824829
validation rsquare: 0.07597142457962036
Epoch:  110  Trait:  Cond_O , Train_R_square_value:  0.6201549768447876
validat

Epoch:  50  Trait:  Cond_O , Train_R_square_value:  0.3988378643989563
validation rsquare: 0.07427436113357544
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  0.4435391426086426
validation rsquare: 0.11372756958007812
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  0.49833476543426514
validation rsquare: 0.26260095834732056
copying best model
Epoch:  80  Trait:  Cond_O , Train_R_square_value:  0.5161689519882202
validation rsquare: 0.170928955078125
Epoch:  90  Trait:  Cond_O , Train_R_square_value:  0.5097938776016235
validation rsquare: 0.25327014923095703
Epoch:  100  Trait:  Cond_O , Train_R_square_value:  0.47699737548828125
validation rsquare: 0.10868453979492188
Epoch:  110  Trait:  Cond_O , Train_R_square_value:  0.5526697635650635
validation rsquare: 0.3103128671646118
copying best model
Epoch:  120  Trait:  Cond_O , Train_R_square_value:  0.544575572013855
validation rsquare: 0.014866650104522705
Epoch:  130  Trait:  Cond_O , Train_R_square_value:  0.5571840405464172


Epoch:  20  Trait:  Cond_O , Train_R_square_value:  0.18963652849197388
validation rsquare: -0.015842080116271973
Epoch:  30  Trait:  Cond_O , Train_R_square_value:  0.10103267431259155
validation rsquare: -0.01579868793487549
Epoch:  40  Trait:  Cond_O , Train_R_square_value:  0.12853413820266724
validation rsquare: -0.08398926258087158
Epoch:  50  Trait:  Cond_O , Train_R_square_value:  0.16975682973861694
validation rsquare: 0.0222703218460083
copying best model
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  0.2364886999130249
validation rsquare: -0.10493409633636475
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  0.2281358242034912
validation rsquare: -0.08451783657073975
Epoch:  80  Trait:  Cond_O , Train_R_square_value:  0.154746413230896
validation rsquare: -0.005082368850708008
Epoch:  90  Trait:  Cond_O , Train_R_square_value:  0.2435622215270996
validation rsquare: -0.087882399559021
Epoch:  100  Trait:  Cond_O , Train_R_square_value:  0.2546011209487915
validation r

Epoch:  0  Trait:  Cond_O , Train_R_square_value:  -53.954105377197266
validation rsquare: -8.021982192993164
copying best model
Epoch:  10  Trait:  Cond_O , Train_R_square_value:  -0.24106693267822266
validation rsquare: -0.016085147857666016
copying best model
Epoch:  20  Trait:  Cond_O , Train_R_square_value:  -0.0552746057510376
validation rsquare: -0.002935051918029785
copying best model
Epoch:  30  Trait:  Cond_O , Train_R_square_value:  -0.040253639221191406
validation rsquare: -0.00064849853515625
copying best model
Epoch:  40  Trait:  Cond_O , Train_R_square_value:  -0.023929357528686523
validation rsquare: -0.0634927749633789
Epoch:  50  Trait:  Cond_O , Train_R_square_value:  -0.0871354341506958
validation rsquare: -0.2236776351928711
Epoch:  60  Trait:  Cond_O , Train_R_square_value:  -0.05064654350280762
validation rsquare: -0.07219898700714111
Epoch:  70  Trait:  Cond_O , Train_R_square_value:  -0.25480782985687256
validation rsquare: -0.1360405683517456
Epoch:  80  Trait

In [20]:
bestParams = {}
bestParams['LMA_O'] = {}
bestParams['LMA_O']['lr'] = 0.0005
bestParams['Narea_O'] = {}
bestParams['Narea_O']['lr'] = 0.0005
bestParams['SPAD_O'] = {}
bestParams['SPAD_O']['lr'] = 0.0005
bestParams['Nmass_O'] = {}
bestParams['Nmass_O']['lr'] = 0.0005
bestParams['Pmass_O'] = {}
bestParams['Pmass_O']['lr'] = 0.0005
bestParams['Photo_O'] = {}
bestParams['Photo_O']['lr'] = 0.0005
bestParams['Vcmax'] = {}
bestParams['Vcmax']['lr'] = 0.0001
bestParams['J'] = {}
bestParams['J']['lr'] = 0.0001
bestParams['Vcmax25'] = {}
bestParams['Vcmax25']['lr'] = 0.0005
bestParams['Cond_O'] = {}
bestParams['Cond_O']['lr'] = 0.0005



## Test model

In [30]:
test_results = {}
for target_trait in run_traits:

    test_ds, testloader = load_ds(target_trait, "test", batch_size)

    model = copy.deepcopy(bestModels[target_trait])
    
    Test_R_square_value = checkPerform(model, test_ds, testloader)
    print("Trait: ", target_trait, ", Test_R_square_value: ", np.asscalar(Test_R_square_value))
    test_results[target_trait] = Test_R_square_value

Trait:  LMA_O , Test_R_square_value:  0.8323739171028137
Trait:  Narea_O , Test_R_square_value:  0.8424593210220337
Trait:  SPAD_O , Test_R_square_value:  0.8378594517707825
Trait:  Nmass_O , Test_R_square_value:  0.5900506973266602
Trait:  Pmass_O , Test_R_square_value:  0.4915919303894043
Trait:  Vcmax , Test_R_square_value:  0.6528968811035156
Trait:  Vcmax25 , Test_R_square_value:  0.5315710306167603
Trait:  J , Test_R_square_value:  0.7535698413848877
Trait:  Photo_O , Test_R_square_value:  0.6257905960083008
Trait:  Cond_O , Test_R_square_value:  0.20482397079467773


## Tuning dropout, weight_decay and stddev

In [22]:
run_traits = ["LMA_O", "Narea_O", "SPAD_O"]

params = {
    'dropout': [0., 0.3, 0.5, 0.7], # default 0
    'weight_decay': [0., 0.0001, 0.0003, 0.0005], # default 0
    'stddev' : [0., 0.01, 0.05] # default 0
}
bestModels = {}
train_results = {}
for target_trait in run_traits:
    best_validation_rsquare = -99999.99
    best_dropout, best_weight_decay, best_stddev = 0., 0., 0.0
    lr = bestParams[target_trait]['lr']
    for dropout in params['dropout']:
        for weight_decay in params['weight_decay']:
            for stddev in params['stddev']:
                print("********** grid search ********** ")
                print (" dropout: " + str(dropout) + " weight_decay: " + str(weight_decay) + " stddev: "+str(stddev))
                print("********************************* ")
                lr = bestParams[target_trait]['lr']
                trained_model, validation_rsquare = train_model(trait=target_trait, structure="model1", 
                                                         dropout=dropout, lr=lr, 
                                                         weight_decay=weight_decay,
                                                         stddev=stddev)
                if best_validation_rsquare < validation_rsquare:
                    best_model = copy.deepcopy(trained_model)
                    best_validation_rsquare = validation_rsquare
                    best_dropout, best_weight_decay,best_stddev = dropout, weight_decay, stddev

                    
    print("gridsearch: best_validation_rsquare: " + str(np.asscalar(best_validation_rsquare)))
    print("Best parameters: ", best_dropout, best_weight_decay, best_stddev)
    bestModels[target_trait] = copy.deepcopy(best_model)
    train_results[target_trait] =  best_validation_rsquare
    bestParams[target_trait]["dropout"], bestParams[target_trait]["weight_decay"], bestParams[target_trait]["stddev"]= best_dropout, best_weight_decay, best_stddev


********** grid search ********** 
 dropout: 0.0 weight_decay: 0.0 stddev: 0.0
********************************* 
HyperSpectrumModelAvgPoolDeepCNN_V(
  (noise): GaussianNoise()
  (avgPool1): AvgPool1d(kernel_size=(5,), stride=(5,), padding=(0,))
  (block1): DeepBasicBlock(
    (Conv1D): Conv1d(1, 50, kernel_size=(5,), stride=(5,), padding=(1,))
    (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block2_1): DeepBasicBlock(
    (Conv1D): Conv1d(50, 100, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block2_2): DeepBasicBlock(
    (Conv1D): Conv1d(100, 200, kernel_size=(3,), stride=(3,))
    (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU6(inplace)
  )
  (block3_1): DeepBasicBlock(
    (Conv1D): Conv1d(200, 400, kernel_size=(3,), stride=(3,))
    (bn): Bat

Epoch:  210  Trait:  LMA_O , Train_R_square_value:  0.9348336458206177
validation rsquare: 0.8574291467666626
Epoch:  220  Trait:  LMA_O , Train_R_square_value:  0.9479725360870361
validation rsquare: 0.8403083086013794
Epoch:  230  Trait:  LMA_O , Train_R_square_value:  0.9434458613395691
validation rsquare: 0.8211070895195007
Epoch:  240  Trait:  LMA_O , Train_R_square_value:  0.9294744729995728
validation rsquare: 0.8260141015052795
Epoch:  250  Trait:  LMA_O , Train_R_square_value:  0.9440353512763977
validation rsquare: 0.807477593421936
Epoch:  260  Trait:  LMA_O , Train_R_square_value:  0.9174656867980957
validation rsquare: 0.7548341155052185
Epoch:  270  Trait:  LMA_O , Train_R_square_value:  0.9454821348190308
validation rsquare: 0.8262419700622559
model is not improved in 20
best_validation_rsquare 0.8637473583221436
********** grid search ********** 
 dropout: 0.0 weight_decay: 0.0 stddev: 0.05
********************************* 
HyperSpectrumModelAvgPoolDeepCNN_V(
  (noise)

Epoch:  0  Trait:  LMA_O , Train_R_square_value:  0.2151116132736206
validation rsquare: -0.0012601613998413086
copying best model
Epoch:  10  Trait:  LMA_O , Train_R_square_value:  0.8408172130584717
validation rsquare: 0.5697087645530701
copying best model
Epoch:  20  Trait:  LMA_O , Train_R_square_value:  0.8692220449447632
validation rsquare: 0.6938728094100952
copying best model
Epoch:  30  Trait:  LMA_O , Train_R_square_value:  0.894206702709198
validation rsquare: -0.5496667623519897
Epoch:  40  Trait:  LMA_O , Train_R_square_value:  0.8664603233337402
validation rsquare: -0.4557321071624756
Epoch:  50  Trait:  LMA_O , Train_R_square_value:  0.8910388946533203
validation rsquare: 0.6210435628890991
Epoch:  60  Trait:  LMA_O , Train_R_square_value:  0.9297409057617188
validation rsquare: 0.5718084573745728
Epoch:  70  Trait:  LMA_O , Train_R_square_value:  0.9072932600975037
validation rsquare: 0.7172122597694397
copying best model
Epoch:  80  Trait:  LMA_O , Train_R_square_value

Epoch:  40  Trait:  LMA_O , Train_R_square_value:  0.8630427122116089
validation rsquare: 0.7157897353172302
Epoch:  50  Trait:  LMA_O , Train_R_square_value:  0.8762410283088684
validation rsquare: 0.8203482627868652
copying best model
Epoch:  60  Trait:  LMA_O , Train_R_square_value:  0.8853100538253784
validation rsquare: 0.8293620347976685
copying best model
Epoch:  70  Trait:  LMA_O , Train_R_square_value:  0.8780432939529419
validation rsquare: 0.7878997325897217
Epoch:  80  Trait:  LMA_O , Train_R_square_value:  0.8916339874267578
validation rsquare: 0.8195022940635681
Epoch:  90  Trait:  LMA_O , Train_R_square_value:  0.8949829339981079
validation rsquare: 0.8234829902648926
Epoch:  100  Trait:  LMA_O , Train_R_square_value:  0.8995459079742432
validation rsquare: 0.8326601982116699
copying best model
Epoch:  110  Trait:  LMA_O , Train_R_square_value:  0.8890897631645203
validation rsquare: 0.8089033365249634
Epoch:  120  Trait:  LMA_O , Train_R_square_value:  0.914258480072021

Epoch:  270  Trait:  LMA_O , Train_R_square_value:  0.848098635673523
validation rsquare: 0.7952596545219421
Epoch:  280  Trait:  LMA_O , Train_R_square_value:  0.8610454797744751
validation rsquare: 0.7909923195838928
Epoch:  290  Trait:  LMA_O , Train_R_square_value:  0.8312522768974304
validation rsquare: 0.8117122650146484
Epoch:  300  Trait:  LMA_O , Train_R_square_value:  0.8571217656135559
validation rsquare: 0.8227481842041016
copying best model
Epoch:  310  Trait:  LMA_O , Train_R_square_value:  0.8744898438453674
validation rsquare: 0.7922154068946838
Epoch:  320  Trait:  LMA_O , Train_R_square_value:  0.85994553565979
validation rsquare: 0.79329514503479
Epoch:  330  Trait:  LMA_O , Train_R_square_value:  0.8280053734779358
validation rsquare: 0.7512575387954712
Epoch:  340  Trait:  LMA_O , Train_R_square_value:  0.8571463823318481
validation rsquare: 0.8126699924468994
Epoch:  350  Trait:  LMA_O , Train_R_square_value:  0.8657774329185486
validation rsquare: 0.8223453164100

...

Epoch:  450  Trait:  SPAD_O , Train_R_square_value:  0.8042142391204834
validation rsquare: 0.8442429304122925
Epoch:  460  Trait:  SPAD_O , Train_R_square_value:  0.8055601119995117
validation rsquare: 0.8774664402008057
Epoch:  470  Trait:  SPAD_O , Train_R_square_value:  0.8053805828094482
validation rsquare: 0.8624714016914368
Epoch:  480  Trait:  SPAD_O , Train_R_square_value:  0.8017628788948059
validation rsquare: 0.8577609062194824
Epoch:  490  Trait:  SPAD_O , Train_R_square_value:  0.8297457695007324
validation rsquare: 0.8314552903175354
Epoch:  500  Trait:  SPAD_O , Train_R_square_value:  0.8244779109954834
validation rsquare: 0.8665386438369751
Epoch:  510  Trait:  SPAD_O , Train_R_square_value:  0.8016252517700195
validation rsquare: 0.8606197834014893
Epoch:  520  Trait:  SPAD_O , Train_R_square_value:  0.8076777458190918
validation rsquare: 0.8589656352996826
Epoch:  530  Trait:  SPAD_O , Train_R_square_value:  0.8095971941947937
validation rsquare: 0.8498033881187439
E

Epoch:  400  Trait:  SPAD_O , Train_R_square_value:  0.8794679641723633
validation rsquare: 0.8578751087188721
Epoch:  410  Trait:  SPAD_O , Train_R_square_value:  0.843626081943512
validation rsquare: 0.8219047784805298
Epoch:  420  Trait:  SPAD_O , Train_R_square_value:  0.8676337003707886
validation rsquare: 0.8353734016418457
Epoch:  430  Trait:  SPAD_O , Train_R_square_value:  0.8696610331535339
validation rsquare: 0.8130616545677185
Epoch:  440  Trait:  SPAD_O , Train_R_square_value:  0.8728464245796204
validation rsquare: 0.8110522627830505
Epoch:  450  Trait:  SPAD_O , Train_R_square_value:  0.884628176689148
validation rsquare: 0.8087336421012878
Epoch:  460  Trait:  SPAD_O , Train_R_square_value:  0.8803887963294983
validation rsquare: 0.8329755067825317
Epoch:  470  Trait:  SPAD_O , Train_R_square_value:  0.8868495225906372
validation rsquare: 0.7793982028961182
Epoch:  480  Trait:  SPAD_O , Train_R_square_value:  0.8851376175880432
validation rsquare: 0.8085463047027588
Epo

Epoch:  0  Trait:  SPAD_O , Train_R_square_value:  -0.025557398796081543
validation rsquare: -0.04102599620819092
copying best model
Epoch:  10  Trait:  SPAD_O , Train_R_square_value:  0.6339981555938721
validation rsquare: 0.672539234161377
copying best model
Epoch:  20  Trait:  SPAD_O , Train_R_square_value:  0.7080550789833069
validation rsquare: 0.8071484565734863
copying best model
Epoch:  30  Trait:  SPAD_O , Train_R_square_value:  0.6847659349441528
validation rsquare: 0.8104889988899231
copying best model
Epoch:  40  Trait:  SPAD_O , Train_R_square_value:  0.7085332274436951
validation rsquare: 0.8132044076919556
copying best model
Epoch:  50  Trait:  SPAD_O , Train_R_square_value:  0.7475788593292236
validation rsquare: 0.8345475196838379
copying best model
Epoch:  60  Trait:  SPAD_O , Train_R_square_value:  0.7640577554702759
validation rsquare: 0.8148292899131775
Epoch:  70  Trait:  SPAD_O , Train_R_square_value:  0.7562323808670044
validation rsquare: 0.7957859039306641
Epo

Epoch:  50  Trait:  SPAD_O , Train_R_square_value:  0.8082683682441711
validation rsquare: 0.8052654266357422
copying best model
Epoch:  60  Trait:  SPAD_O , Train_R_square_value:  0.822845458984375
validation rsquare: 0.7691947221755981
Epoch:  70  Trait:  SPAD_O , Train_R_square_value:  0.8042151927947998
validation rsquare: 0.8578444123268127
copying best model
Epoch:  80  Trait:  SPAD_O , Train_R_square_value:  0.7878262996673584
validation rsquare: 0.8729113936424255
copying best model
Epoch:  90  Trait:  SPAD_O , Train_R_square_value:  0.7951177954673767
validation rsquare: 0.8503800630569458
Epoch:  100  Trait:  SPAD_O , Train_R_square_value:  0.8298293352127075
validation rsquare: 0.8486269116401672
Epoch:  110  Trait:  SPAD_O , Train_R_square_value:  0.8333626985549927
validation rsquare: 0.8209583163261414
Epoch:  120  Trait:  SPAD_O , Train_R_square_value:  0.8135780096054077
validation rsquare: 0.7637026906013489
Epoch:  130  Trait:  SPAD_O , Train_R_square_value:  0.837916

Epoch:  350  Trait:  SPAD_O , Train_R_square_value:  0.8413786888122559
validation rsquare: 0.865885317325592
Epoch:  360  Trait:  SPAD_O , Train_R_square_value:  0.8488481044769287
validation rsquare: 0.8644625544548035
Epoch:  370  Trait:  SPAD_O , Train_R_square_value:  0.8515540361404419
validation rsquare: 0.8483604788780212
Epoch:  380  Trait:  SPAD_O , Train_R_square_value:  0.8628655076026917
validation rsquare: 0.8673908710479736
Epoch:  390  Trait:  SPAD_O , Train_R_square_value:  0.850200891494751
validation rsquare: 0.8589513897895813
Epoch:  400  Trait:  SPAD_O , Train_R_square_value:  0.8363183736801147
validation rsquare: 0.8535745143890381
Epoch:  410  Trait:  SPAD_O , Train_R_square_value:  0.857851505279541
validation rsquare: 0.8497800230979919
Epoch:  420  Trait:  SPAD_O , Train_R_square_value:  0.8664126396179199
validation rsquare: 0.858137845993042
Epoch:  430  Trait:  SPAD_O , Train_R_square_value:  0.844855546951294
validation rsquare: 0.8441027402877808
Epoch:

In [ ]:
val_results = {}
criterion = nn.MSELoss()
for target_trait in run_traits:

    val_ds, validationloader = load_ds(target_trait, "val", batch_size)
    val_label_sq_diff = compute_label_sq_diff(val_ds)

    model = copy.deepcopy(bestModels[target_trait])
    pred_sq_diff = 0.0

    num_iters = 0
    running_loss = 0.0
    model.eval()
    for i, data in enumerate(validationloader, 0):
        values, labels = data
        labels = labels.unsqueeze(1)
        values = values.to(device)
        labels = labels.to(device)
        outputs = model(values)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        num_iters += 1
        diff = outputs - labels
        #diff = diff.cpu()
        pred_sq_diff += torch.sum(torch.mul(diff, diff), 0).detach().cpu().numpy()

    print("Trait: ", target_trait, ", test loss: ", running_loss / num_iters)
    Val_R_square_value = 1 - pred_sq_diff / val_label_sq_diff
    print("Trait: ", target_trait, ", Val_R_square_value: ", np.asscalar(Val_R_square_value))
    val_results[target_trait] = Val_R_square_value

## Test model

In [ ]:
test_results = {}
for target_trait in run_traits:

    test_ds, testloader = load_ds(target_trait, "test", batch_size)

    model = copy.deepcopy(bestModels[target_trait])
    
    Test_R_square_value = checkPerform(model, test_ds, testloader)
    print("Trait: ", target_trait, ", Test_R_square_value: ", np.asscalar(Test_R_square_value))
    test_results[target_trait] = Test_R_square_value

In [ ]:
writer.close()

result_file = open(result_file_name,"w")
result_file.write("-------------------------------------------------\n")
result_file.write("CNN Architecture: "+exp_desc+"\n")
result_file.write("Best paramaters: \n")
for target_trait in run_traits:
    best_lr, best_dropout, best_weight_decay, best_stddev = bestParams[target_trait] 
    result_file.write("trait: " + target_trait +", lr: " + str(best_lr) +", dropout: " 
                      + str(best_dropout)+ ", weight_decay: " + str(best_weight_decay)
                      +", stddev:" +str(best_stddev)+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the training data set results:\n")
total = 0.0
for key, value in train_results.items():
    result_file.write("trait: " + str(key) +", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(train_results))+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the validation data set results: \n")
total = 0.0
for key, value in val_results.items():
    result_file.write("trait: " + str(key) +", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(val_results))+"\n")
result_file.write("-------------------------------------------------\n")
result_file.write("Here are the testing data set results: \n")
total = 0.0
for key, value in test_results.items():
    result_file.write("trait: " + str(key) + ", R square: " + str(np.asscalar(value))+"\n")
    total += np.asscalar(value)
result_file.write("Average R square: " + str(total / len(test_results)) + "\n")
result_file.write("-------------------------------------------------\n")
result_file.close()



result_file = open(result_file_name+"csv","w")
result_file.write("Model, Dataset, LMA_O, Narea_O, SPAD_O, Nmass_O, Parea_O, Pmass_O, Vcmax, Vcmax25, J, Photo_O, Cond_O, Average\n")

result_file.write(exp_desc +", train, ") 
total = 0.0

for trait in all_traits:
    if trait in train_results:
        result_file.write(str(np.asscalar(train_results[trait]))+", ")
        total += np.asscalar(train_results[trait])
    else:
        result_file.write(", ")

result_file.write(str(total/len(train_results))+"\n")

result_file.write(exp_desc +", val, ") 
total = 0.0

for trait in all_traits:
    if trait in val_results:
        result_file.write(str(np.asscalar(val_results[trait]))+", ")
        total += np.asscalar(val_results[trait])
    else:
        result_file.write(", ")

result_file.write(str(total/len(val_results))+"\n")

result_file.write(exp_desc +", test, ") 
total = 0.0

for trait in all_traits:
    if trait in test_results:
        result_file.write(str(np.asscalar(test_results[trait]))+", ")
        total += np.asscalar(test_results[trait])
    else:
        result_file.write(", ")

result_file.write(str(total/len(test_results))+"\n")


result_file.close()


## Train Muiltiple models